In [43]:
import time
import requests
import pandas as pd
from tqdm import tqdm

import chromedriver_binary
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [47]:
# Specify the path to the ChromeDriver executable
chrome_driver_path = r'C:\STUDY\DataScience_Course\Web_Scraping\Web_driver\chromedriver-win64\chromedriver.exe'

# Create a new Service instance with the specified path
service = Service(chrome_driver_path)

# Initialize Chrome WebDriver with the service
browser = webdriver.Chrome(service=service)

browser.get('https://stockmages.netlify.app')

### 1. Srolling to reach the bottom

In [46]:
744000/400

1860.0

In [48]:
for i in tqdm(range(0,744000,400)):
    
    browser.execute_script("window.scrollTo(0,"  + str(i) + ")")
    time.sleep(.1)

100%|██████████████████████████████████████████████████████████████████████████████| 1860/1860 [03:52<00:00,  8.00it/s]


### 2. Scraping the Page Course

In [49]:
soup = BeautifulSoup(browser.page_source, 'html.parser')

### 3. Checking number of images that has to be scraped

In [50]:
len(soup.find_all('div', class_ = 'container'))

9104

### 4. Scraping the Image Details

In [51]:
data = []

for sp in tqdm(soup.find_all('div', class_ = 'container')):

    img_link = sp.find('img').get('src')
    tags     = sp.find('span', class_ = 'tag-color').text[7:].strip()
    likes    = int(sp.find('div', class_ = 'likes-comments').find_all('span')[0].text.strip()[:-6])
    comments = int(sp.find('div', class_ = 'likes-comments').find_all('span')[1].text.strip()[:-9])
    
    data.append([img_link, tags, likes, comments])

100%|███████████████████████████████████████████████████████████████████████████| 9104/9104 [00:00<00:00, 13237.08it/s]


### 5. Saving data in a DataFrame

In [52]:
df = pd.DataFrame(data, columns = ['img_link', 'tags', 'likes', 'comments'])

df.head()

,img_link,tags,likes,comments
0,https://cdn.pixabay.com/photo/2022/03/06/05/30...,"Clouds, Sky, Atmosphere, Blue Sky",196,55
1,https://cdn.pixabay.com/photo/2022/04/07/11/45...,"Bird, Ornithology, Hummingbird",76,20
2,https://cdn.pixabay.com/photo/2022/02/28/15/28...,"Sea, Rainbow, Rainfall, Subtropical",282,106
3,https://cdn.pixabay.com/photo/2022/04/04/02/52...,"Cherry Blossoms, Road, Japan, Sakura",42,11
4,https://cdn.pixabay.com/photo/2022/04/09/18/06...,"Cape Marguerite, Flower, Plant",39,15


### 6. Checking Null Values

In [53]:
df.isnull().sum()

img_link    0
tags        0
likes       0
comments    0
dtype: int64

### 7. Checking Duplicate Values

In [54]:
df['img_link'].describe()

count                                                  9104
unique                                                 9088
top       https://cdn.pixabay.com/photo/2022/03/06/05/30...
freq                                                      2
Name: img_link, dtype: object

### 8. Removing Duplicate Values

In [55]:
df = df.drop_duplicates()

### 9. Saving the DataFrame as CSV

In [56]:
df.to_csv('C:\STUDY\DataScience_Course\Web_Scraping\images.csv', index = False)